In [1]:
import sys
import math
import mmap
import posix_ipc as ipc
import torch

from io import BytesIO

In [4]:
def save_to_shared_memory(filename, name):
    with open(filename, "rb") as f:
        obj = f.read()
    size = sys.getsizeof(obj)
    pages = math.ceil(size / ipc.PAGE_SIZE)
    try:
        memory = ipc.SharedMemory(name=name, flags=ipc.O_CREAT, size=ipc.PAGE_SIZE * pages)
    except ValueError: # retry once more
        memory = ipc.SharedMemory(name=name, flags=ipc.O_CREAT, size=ipc.PAGE_SIZE * pages)
    io = mmap.mmap(fileno=memory.fd, length=memory.size, flags=mmap.MAP_SHARED, 
                   prot=mmap.PROT_WRITE)
    io.write(obj)
    io.close()
    memory.close_fd()
    return memory.name

save_to_shared_memory("data/preprocessed/vocab.pkl", "vocab.pkl")
save_to_shared_memory("data/preprocessed/train_ds.pkl", "train_ds.pkl")

'train_ds.pkl'

In [5]:
def read_from_shared_memory(name):
    memory = ipc.SharedMemory(name=name)
    io = mmap.mmap(memory.fd, memory.size, mmap.MAP_SHARED, mmap.PROT_READ)
    bytes_obj = BytesIO(io.read())
    io.close()
    memory.close_fd()
    return bytes_obj

test = torch.load(read_from_shared_memory("vocab.pkl"))

In [23]:
def release_process_from_shared_memory(memory_identifier):
    try:
        shared_mem = ipc.SharedMemory(memory_identifier)
        shared_mem.unlink()
        # will be destructed if every process `unlinked`
        print("INFO: shared memory detached from process")
    except ipc.ExistentialError:
        print("WARN: shared memory not found")
        pass

release_process_from_shared_memory("vocab.pkl")

WARN: shared memory not found
